In [2]:
import os
import numpy as np

import itk
from itk import TubeTK as ttk

from itkwidgets import view

In [3]:
main_fld = '/media/peirong/PR/IXI/IXI-Vessel'
pre_fld = os.path.join(main_fld, 'level-1')
cur_fld = os.path.join(main_fld, 'level-2')

In [4]:
ImageType = itk.Image[itk.F, 3]

im1iso = itk.imread(os.path.join(pre_fld, "MRA-VesselEnhanced.mha"))
im1BrainVess = itk.imread(os.path.join(pre_fld, "MRA-Brain-VesselEnhanced.mha"))

In [5]:
imMath = ttk.ImageMath.New(im1BrainVess)
imMath.MedianFilter(1)
imMath.Threshold(0.0000000001, 1, 1, 0)
im1VessMask = imMath.GetOutputShort()

ccSeg = ttk.SegmentConnectedComponents.New(im1VessMask)
ccSeg.SetMinimumVolume(4)
ccSeg.Update()
im1VessMaskCC = ccSeg.GetOutput()

In [6]:
imMathSS = ttk.ImageMath.New(im1VessMaskCC)
imMath.Threshold(1,1,0,1)
im1VessMaskInv = imMath.GetOutputFloat()
distFilter = itk.DanielssonDistanceMapImageFilter.New(im1VessMaskInv)
distFilter.Update()
im1SeedRadius = distFilter.GetOutput()

view(im1SeedRadius)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [9]:
numSeeds = 20

vSeg = ttk.SegmentTubes.New(Input=im1iso)
#vSeg.SetVerbose(True)
vSeg.SetMinCurvature(0.0)
vSeg.SetMinRoundness(0.0)
vSeg.SetMinRidgeness(0.8)
vSeg.SetMinLevelness(0.0)
vSeg.SetRadiusInObjectSpace( 0.75 )
vSeg.SetBorderInIndexSpace(3)
vSeg.SetSeedMask( im1SeedRadius )
vSeg.SetSeedRadiusMask( im1SeedRadius )
#NOTE: Origianl code: vSeg.SetOptimizeRadius(False) 
vSeg.SetUseSeedMaskAsProbabilities(True)
#NOTE: Origianl code: vSeg.SetSeedExtractionMinimumProbability(0.4)
#vSeg.SetSeedMaskMaximumNumberOfPoints( numSeeds )
#vSeg.SetDynamicScale(False)
vSeg.ProcessSeeds()

AttributeError: 'tubeSegmentTubesIF3' object has no attribute 'SetUseSeedMaskAsProbabilities'

In [ ]:

vSeg = ttk.SegmentTubes[ImageType].New()
vSeg.SetInput( im1iso )
vSeg.SetVerbose( False )
vSeg.SetRadiusInObjectSpace( 0.33 )
vSeg.SetMinRoundness(0.2)
vSeg.SetDynamicScale(False)
tube = vSeg.ExtractTubeInObjectSpace( xp, 1 )
tubeG = vSeg.GetTubeGroup()


TubesToImageFilterType = ttk.ConvertTubesToImage[ImageType]
tubesToImageFilter = TubesToImageFilterType.New()
tubesToImageFilter.SetUseRadius(True)
tubesToImageFilter.SetTemplateImage(im1)
tubesToImageFilter.SetInput(tubeG)
tubesToImageFilter.Update()


In [ ]:
tubeMaskImage = vSeg.GetTubeMaskImage()
view(tubeMaskImage)

In [ ]:
SOWriter = itk.SpatialObjectWriter[3].New(vSeg.GetTubeGroup())
SOWriter.SetFileName( os.path.join(cur_fld, "MRA-Vessels.tre") )
SOWriter.Update()